## Задача предсказания оттока клиентов телеком компании

задача бинарной классификации

$$g(x_i) \approx y_i$$

$x_i$ - вектор признаков по клиенту i

$y_i \in \{ 0, 1 \}$ 

0 - No Churn
1 - Churn

- `!wget` - команда Linux для загрузки данных
- `pd.read_csv()` - чтение csv файлов
- `df.head()` - просмотр первых строк датафрейма
- `df.head().T` - просмотр транспонированного датафрейма
- `df.columns` - получение названий столбцов датафрейма
- `df.columns.str.lower()` - приведение всех букв к нижнему регистру
- `df.columns.str.replace(' ', '_')` - замена пробелов на подчеркивание
- `df.dtypes` - получение типов данных всех столбцов
- `df.index` - получение индексов датафрейма
- `pd.to_numeric()` - преобразование значений столбца в числовой формат. Аргумент `errors='coerce'` позволяет выполнить преобразование, несмотря на ошибки.
- `df.fillna()` - замена NA значений на определённое значение
- `(df.x == "yes").astype(int)` - преобразование столбца x с yes-no значениями в числовой формат.


In [1]:
import pandas as pd
import numpy as np
 
import matplotlib.pyplot as plt
 
# data = "https://..."
# !wget $data -O data-week-3.csv

In [2]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [5]:
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


## Подготовка данных

In [4]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)

In [6]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [7]:
df.churn = (df.churn == 'yes').astype(int)

## Exploratory data analysis

Анализ данных (Exploratory data analysis, EDA) для этого проекта включал:

- Проверку пропущенных значений
- Изучение распределения целевой переменной (отток)
- Рассмотрение числовых и категориальных переменных

Полезные функции:

- `df.isnull().sum()` - возвращает количество пропущенных значений в датафрейме.
- `df.x.value_counts()` - возвращает количество значений для каждой категории в столбце `x`. Аргумент `normalize=True` позволяет получить процент для каждой категории. В этом проекте среднее значение оттока равно уровню оттока, полученному с помощью метода `value_counts`.
- `round(x, y)` - округляет число `x` до `y` знаков после запятой.
- `df[x].nunique()` - возвращает количество уникальных значений в столбце `x`.


In [10]:
df.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [11]:
df.churn.value_counts()

churn
0    5174
1    1869
Name: count, dtype: int64

Среди них 1869 — недовольные клиенты (отток), а остальные 5174 — довольные клиенты (не отток). Понимание распределения целевой переменной — важный шаг в любой задаче анализа данных или моделирования, поскольку оно дает ценную информацию о балансе классов данных, которая может повлиять на решения по моделированию и метрики оценки.



In [12]:
df.churn.value_counts(normalize=True)

churn
0    0.73463
1    0.26537
Name: proportion, dtype: float64

In [13]:
global_churn_rate = df.churn.mean()
round(global_churn_rate, 2)

0.27

In [14]:
numerical_vars = df.select_dtypes(include=['int64', 'float64'])
categorical_vars = df.select_dtypes(include=['object'])
 
print("Numerical Variables:")
print(numerical_vars.columns)
 
print("\nCategorical Variables:")
print(categorical_vars.columns)

Numerical Variables:
Index(['seniorcitizen', 'tenure', 'monthlycharges', 'churn'], dtype='object')

Categorical Variables:
Index(['customerid', 'gender', 'partner', 'dependents', 'phoneservice',
       'multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup',
       'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies',
       'contract', 'paperlessbilling', 'paymentmethod', 'totalcharges'],
      dtype='object')


In [17]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']
 
# Удалим 'customerid', 'tenure', 'monthlycharges', 'totalcharges', 'churn'
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
       'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

In [19]:
df[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

## Feature importance

In [20]:
female_mean = df[df.gender == 'female'].churn.mean()
print('gender == female:', round(female_mean, 3))

male_mean = df[df.gender == 'male'].churn.mean()
print('gender == male:  ', round(male_mean, 3))

gender == female: 0.269
gender == male:   0.262


In [21]:
partner_yes = df[df.partner == 'yes'].churn.mean()
print('partner == yes:', round(partner_yes, 3))

partner_no = df[df.partner == 'no'].churn.mean()
print('partner == no :', round(partner_no, 3))

partner == yes: 0.197
partner == no : 0.33


- нет большого влияния пола на отток клиента
- наличие партнера оказывает большее влияние на отток клиента, наличие партнера уменьшает вероятность ухода клиента

0.06872084081133259

### Risk ratio

В контексте машинного обучения и классификации «коэффициент риска» обычно относится к статистической мере, используемой для оценки вероятности или вероятности определенного события, происходящего в одной группе по сравнению с другой. 


- Определение коэффициента риска: коэффициент риска (относительный риск) определяется как вероятность события, происходящего в одной группе, деленная на вероятность того же события, происходящего в другой группе. В случае коэффициента оттока вы обычно сравниваете две группы: одну группу, которая демонстрирует определенную характеристику или поведение (например, клиент ушел), и другую группу, которая не демонстрирует эту характеристику (например, клиент не ушел).
- Интерпретация: Коэффициент риска больше 1 предполагает, что событие (в данном случае отток) более вероятно в первой группе по сравнению со второй группой. Коэффициент риска меньше 1 предполагает, что событие менее вероятно в первой группе. Коэффициент риска, равный 1, означает, что нет никакой разницы в риске между двумя группами.
- Применение: Мы можем использовать коэффициенты риска для оценки влияния различных функций или вмешательств на отток. Например, мы можем рассчитать коэффициент риска оттока для клиентов, которые получили рекламное предложение, по сравнению с теми, кто его не получил. Если коэффициент риска значительно больше 1, это означает, что рекламное предложение оказало положительное влияние на снижение оттока.
- Статистическая значимость: важно также учитывать статистическую значимость при интерпретации коэффициентов риска. Статистические тесты, такие как тесты хи-квадрат или доверительные интервалы, могут помочь определить, являются ли наблюдаемые различия в показателях оттока статистически значимыми.

In [24]:
print(partner_yes / global_churn_rate)

 
print(partner_no / global_churn_rate)

0.7410375164075894
1.241963847619165


### 1. Diff

**global - group**

- Если результат **< 0**:  
  _Более вероятен отток_
  
- Если результат **> 0**:  
  _Менее вероятен отток_

### 2. Risk ratio

**Формула:**


$\text{Risk} = \frac{\text{group}}{\text{global}}$


- Если результат **> 1**:  
  _Более вероятен отток_
  
- Если результат **< 1**:  
  _Менее вероятен отток_




```sql
SELECT
    gender,
    AVG(churn),
    AVG(churn) - global_churn AS diff,
    AVG(churn) / global_churn AS risk
FROM
    date
GROUP BY
    gender;

Мы можем выполнить этот анализ для всех переменных, а не только для переменной пола.

In [25]:
df_group = df.groupby(by='gender').churn.agg(['mean'])
df_group['diff'] = df_group['mean'] - global_churn_rate
df_group['risk'] = df_group['mean'] / global_churn_rate
df_group

,mean,diff,risk
gender,,,
female,0.269209,0.003839,1.014466
male,0.261603,-0.003766,0.985807


In [26]:
from IPython.display import display

for col in categorical:
    df_group = df.groupby(by=col).churn.agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_churn_rate
    df_group['risk'] = df_group['mean'] / global_churn_rate
    display(df_group)

,mean,diff,risk
gender,,,
female,0.269209,0.003839,1.014466
male,0.261603,-0.003766,0.985807


,mean,diff,risk
seniorcitizen,,,
0,0.236062,-0.029308,0.889557
1,0.416813,0.151443,1.570686


,mean,diff,risk
partner,,,
no,0.329580,0.064210,1.241964
yes,0.196649,-0.068721,0.741038


,mean,diff,risk
dependents,,,
no,0.312791,0.047422,1.178700
yes,0.154502,-0.110868,0.582215


,mean,diff,risk
phoneservice,,,
no,0.249267,-0.016103,0.939319
yes,0.267096,0.001726,1.006506


...

,mean,diff,risk
streamingtv,,,
no,0.335231,0.069861,1.263261
no_internet_service,0.074050,-0.191320,0.279044
yes,0.300702,0.035332,1.133143


,mean,diff,risk
streamingmovies,,,
no,0.336804,0.071434,1.269188
no_internet_service,0.074050,-0.191320,0.279044
yes,0.299414,0.034044,1.128291


,mean,diff,risk
contract,,,
month-to-month,0.427097,0.161727,1.609440
one_year,0.112695,-0.152675,0.424672
two_year,0.028319,-0.237051,0.106714


,mean,diff,risk
paperlessbilling,,,
no,0.163301,-0.102069,0.615371
yes,0.335651,0.070281,1.264842


,mean,diff,risk
paymentmethod,,,
bank_transfer_(automatic),0.167098,-0.098271,0.629681
credit_card_(automatic),0.152431,-0.112939,0.574410
electronic_check,0.452854,0.187484,1.706502
mailed_check,0.191067,-0.074303,0.720003
